## **LIBRARIES**

In [0]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Reshape, UpSampling2D
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.layers import Activation, Dense, Flatten
import matplotlib as plt
import graphviz
import cv2
import os
from skimage import io
import numpy as np
import pandas as pd
import numpy.random as rng
from sklearn.utils import shuffle
import zipfile
import io
import tensorflow as tf
import keras 
import pandas as pd
from os import path
import shutil
from tqdm import tqdm,trange
import sys
import gc
from numpy.random import choice
from itertools import combinations
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import math
from tqdm import tqdm,tnrange, tqdm_notebook
import keras as K
import matplotlib.pyplot as plt
from collections import Counter
from keras.utils import np_utils
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import Activation,BatchNormalization,Lambda, Input, Conv2D, Dense,MaxPooling2D, Dropout, Flatten, UpSampling2D, Reshape

Using TensorFlow backend.


In [0]:
!pip install pgmpy

    100% |████████████████████████████████| 296kB 7.4MB/s 
    100% |████████████████████████████████| 1.3MB 15.6MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
import pandas as pd
import numpy as np
import types
from pgmpy.estimators import K2Score
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.readwrite import ProbModelXMLWriter, get_probmodel_data
from pgmpy.sampling import BayesianModelSampling
from pgmpy.sampling import GibbsSampling
from pgmpy.inference import VariableElimination
import time 
import networkx as nx
import matplotlib.pyplot as plt 
from pgmpy.estimators import HillClimbSearch
from pgmpy.inference import BeliefPropagation

In [0]:
from google.colab import files
uploaded = files.upload()

Saving 15features.csv to 15features.csv
Saving merged_data.csv to merged_data.csv
Saving merged_data_testing.csv to merged_data_testing.csv


# TASK 2:


**Importing the files**

In [0]:
data = pd.read_csv('merged_data.csv')
data = data.rename(columns = {'label':'L'})
data
data_1 = data.loc[:,'L':'r_f15']
data_1
c = list(data_1.columns.values)
#print(c)
data_1 = data_1[['l_f1', 'l_f2', 'l_f3', 'l_f4', 'l_f5', 'l_f6', 'l_f7', 'l_f8', 'l_f9', 'l_f10', 'l_f11', 'l_f12', 'l_f13', 'l_f14', 'l_f15', 'r_f1', 'r_f2', 'r_f3', 'r_f4', 'r_f5', 'r_f6', 'r_f7', 'r_f8', 'r_f9', 'r_f10', 'r_f11', 'r_f12', 'r_f13', 'r_f14', 'r_f15','L']]
#data_1 = data_1.fillna(0)
#data_1 = data_1
data_1 = data_1.dropna(how='any',axis=0)
data_1.shape

(111753, 31)

**Data Generation**

In [0]:
def data_gen(filename,save_file):
  import pandas as pd
  data_features = pd.read_csv("15features.csv")
  data_pairs    = pd.read_csv(filename+".csv")
  data_features['left'] = data_features['imagename'] 
  data_features['right'] = data_features['imagename']
  print(data_features.shape)
  print(data_pairs.shape)
  #data_features
  left_merge = pd.merge(data_pairs, data_features, on=['left'], how='left') ## some images are missing in the excel file do 
  print(left_merge.shape)
  #right_merge = pd.merge(data_pairs,data_features,on=['right'],how ='inner')
  #right_merge
  left_merge = left_merge.loc[:, 'left':'right_y']
  left_merge.columns = ["left","right","label","ImageId_l","l_f1","l_f2","l_f3","l_f4","l_f5","l_f6","l_f7","l_f8","l_f9","l_f10","l_f11","l_f12","l_f13","l_f14","l_f15","right_copy"]
  right_merge = pd.merge(left_merge,data_features,on=['right'],how='left')
  print(right_merge.shape)
  right_merge.columns = ["left","right","label","ImageId_l","l_f1","l_f2","l_f3","l_f4","l_f5","l_f6","l_f7","l_f8","l_f9","l_f10","l_f11","l_f12","l_f13","l_f14","l_f15","right_copy",
                         "imageid_r","r_f1","r_f2","r_f3","r_f4","r_f5","r_f6","r_f7","r_f8","r_f9","r_f10","r_f11","r_f12","r_f13","r_f14","r_f15","left_copy"]
  
  merged_data = right_merge.drop(right_merge.columns[[3,19,20,36]], axis=1) 
  merged_data.to_csv(save_file+".csv")

**Creating the models**

In [0]:
def model(list_of_nodes,list_of_edges,i):
    ## ====== Left Network ==========
  model_1 = BayesianModel()
  model_1.add_nodes_from(list_of_nodes)
  model_1.add_edges_from(list_of_edges)
  print("====Model" +str(i)+ " is created====")
  return model_1

**Hill-Climb Search**

In [0]:
data_hill = pd.read_csv('15features.csv')
data_hill.columns = ['Imageid','l_f1', 'l_f2', 'l_f3', 'l_f4', 'l_f5', 'l_f6', 'l_f7', 'l_f8', 'l_f9', 'l_f10', 'l_f11', 'l_f12', 'l_f13', 'l_f14', 'l_f15']
data_hill = data_hill.loc[:,'l_f1':'l_f15']
data_hill.shape
## splitting the data to fit the hill climb search -- I am doing this because the total data is taking more computation time to execute
data_hill_1 = data_1.loc[0:25755]
data_hill_2 = data_1.loc[57195:82622]
data_frames_hill = [data_hill_1,data_hill_2]
data_hill_fit = pd.concat(data_frames_hill,ignore_index=True)


In [0]:
hc = HillClimbSearch(data_hill, scoring_method=K2Score(data_hill))
best_model_HC = hc.estimate()
print("Edges from Hill Climb Search :",best_model_HC.edges())


Edges from Hill Climb Search : [('l_f1', 'l_f5'), ('l_f1', 'l_f2'), ('l_f3', 'l_f7'), ('l_f3', 'l_f1'), ('l_f3', 'l_f12'), ('l_f3', 'l_f2'), ('l_f3', 'l_f13'), ('l_f3', 'l_f9'), ('l_f4', 'l_f3'), ('l_f4', 'l_f6'), ('l_f4', 'l_f7'), ('l_f4', 'l_f1'), ('l_f5', 'l_f10'), ('l_f5', 'l_f13'), ('l_f6', 'l_f13'), ('l_f6', 'l_f2'), ('l_f6', 'l_f9'), ('l_f6', 'l_f10'), ('l_f6', 'l_f5'), ('l_f7', 'l_f6'), ('l_f7', 'l_f8'), ('l_f9', 'l_f1'), ('l_f11', 'l_f15'), ('l_f11', 'l_f14'), ('l_f11', 'l_f4'), ('l_f11', 'l_f12'), ('l_f11', 'l_f6'), ('l_f11', 'l_f3'), ('l_f11', 'l_f10'), ('l_f12', 'l_f6'), ('l_f12', 'l_f13'), ('l_f12', 'l_f5'), ('l_f12', 'l_f7'), ('l_f12', 'l_f9'), ('l_f14', 'l_f4'), ('l_f14', 'l_f10'), ('l_f14', 'l_f3'), ('l_f14', 'l_f12'), ('l_f14', 'l_f15'), ('l_f15', 'l_f10'), ('l_f15', 'l_f2'), ('l_f15', 'l_f4')]


**Caluclating the Corelations **

In [0]:
data_correlation = data_1.loc[:,'l_f1':'l_f15']
data_correlation.corr(method='spearman').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('Set1'), axis=1)

,l_f1,l_f2,l_f3,l_f4,l_f5,l_f6,l_f7,l_f8,l_f9,l_f10,l_f11,l_f12,l_f13,l_f14,l_f15
l_f1,1.0,-0.28,-0.4,-0.47,0.06,0.56,0.35,-0.073,0.053,-0.24,-0.29,0.22,-0.084,-0.17,0.028
l_f2,-0.28,1.0,0.27,0.21,-0.033,-0.41,-0.023,0.0037,-0.045,0.44,0.4,0.2,0.14,0.0022,-0.35
l_f3,-0.4,0.27,1.0,0.71,-0.058,-0.41,-0.076,0.28,-0.13,0.37,0.27,-0.09,0.12,-0.014,-0.11
l_f4,-0.47,0.21,0.71,1.0,-0.06,-0.48,-0.28,0.38,-0.093,0.44,0.13,-0.33,0.26,0.13,0.12
l_f5,0.06,-0.033,-0.058,-0.06,1.0,0.1,0.067,0.029,0.015,0.083,-0.039,0.086,0.074,-0.038,-0.022
l_f6,0.56,-0.41,-0.41,-0.48,0.1,1.0,0.36,-0.077,0.11,-0.22,-0.35,0.34,0.041,-0.23,-0.0066
l_f7,0.35,-0.023,-0.076,-0.28,0.067,0.36,1.0,0.18,0.017,-0.065,-0.073,0.33,0.021,-0.4,-0.34
l_f8,-0.073,0.0037,0.28,0.38,0.029,-0.077,0.18,1.0,-0.037,0.17,-0.34,-0.27,0.4,-0.39,-0.1
l_f9,0.053,-0.045,-0.13,-0.093,0.015,0.11,0.017,-0.037,1.0,0.061,-0.006,0.069,0.1,0.023,0.025
l_f10,-0.24,0.44,0.37,0.44,0.083,-0.22,-0.065,0.17,0.061,1.0,0.26,0.24,0.49,0.017,-0.16


**Creating the Bayesian Models  based on the corelation values **

In [0]:
## MODEL 2
i = 1
nodes_list = ["l_f1" ,"l_f2","l_f3" ,"l_f4","l_f5" ,"l_f6","l_f7" ,"l_f8","l_f9","l_f10","l_f11","l_f12","l_f13","l_f14","l_f15",
              "r_f1" ,"r_f2","r_f3" ,"r_f4","r_f5" ,"r_f6","r_f7" ,"r_f8","r_f9","r_f10","r_f11","r_f12","r_f13","r_f14","r_f15",'L']

edges_list_1 =[('l_f1', 'l_f5'), ('l_f1', 'l_f2'), ('l_f3', 'l_f7'), ('l_f3', 'l_f1'), ('l_f3', 'l_f12'), ('l_f3', 'l_f2'), ('l_f3', 'l_f13'),
               ('l_f3', 'l_f9'), ('l_f4', 'l_f3'), ('l_f4', 'l_f6'), ('l_f4', 'l_f7'), ('l_f4', 'l_f1'), ('l_f5', 'l_f10'), ('l_f5', 'l_f13'), 
               ('l_f6', 'l_f13'), ('l_f6', 'l_f2'), ('l_f6', 'l_f9'), ('l_f6', 'l_f10'), ('l_f6', 'l_f5'), ('l_f7', 'l_f6'), ('l_f7', 'l_f8'), 
               ('l_f9', 'l_f1'), ('l_f11', 'l_f15'), ('l_f11', 'l_f14'), ('l_f11', 'l_f4'), ('l_f11', 'l_f12'), ('l_f11', 'l_f6'), ('l_f11', 'l_f3'),
               ('l_f11', 'l_f10'), ('l_f12', 'l_f6'), ('l_f12', 'l_f13'), ('l_f12', 'l_f5'), ('l_f12', 'l_f7'), ('l_f12', 'l_f9'), ('l_f14', 'l_f4'),
               ('l_f14', 'l_f10'), ('l_f14', 'l_f3'), ('l_f14', 'l_f12'), ('l_f14', 'l_f15'), ('l_f15', 'l_f10'), ('l_f15', 'l_f2'), ('l_f15', 'l_f4'),
               ('l_f2', 'L'),('l_f13', 'L'),('l_f5', 'L'),('l_f10', 'L'),('l_f15', 'L'),
               ('r_f1', 'r_f5'), ('r_f1', 'r_f2'), ('r_f3', 'r_f7'), ('r_f3', 'r_f1'), ('r_f3', 'r_f12'), ('r_f3', 'r_f2'), ('r_f3', 'r_f13'),
               ('r_f3', 'r_f9'), ('r_f4', 'r_f3'), ('r_f4', 'r_f6'), ('r_f4', 'r_f7'), ('r_f4', 'r_f1'), ('r_f5', 'r_f10'), ('r_f5', 'r_f13'), 
               ('r_f6', 'r_f13'), ('r_f6', 'r_f2'), ('r_f6', 'r_f9'), ('r_f6', 'r_f10'), ('r_f6', 'r_f5'), ('r_f7', 'r_f6'), ('r_f7', 'r_f8'), 
               ('r_f9', 'r_f1'), ('r_f11', 'r_f15'), ('r_f11', 'r_f14'), ('r_f11', 'r_f4'), ('r_f11', 'r_f12'), ('r_f11', 'r_f6'), ('r_f11', 'r_f3'),
               ('r_f11', 'r_f10'), ('r_f12', 'r_f6'), ('r_f12', 'r_f13'), ('r_f12', 'r_f5'), ('r_f12', 'r_f7'), ('r_f12', 'r_f9'), ('r_f14', 'r_f4'),
               ('r_f14', 'r_f10'), ('r_f14', 'r_f3'), ('r_f14', 'r_f12'), ('r_f14', 'r_f15'), ('r_f15', 'r_f10'), ('r_f15', 'r_f2'), ('r_f15', 'r_f4'),
               ('r_f2', 'L'),('r_f13', 'L'),('r_f5', 'L'),('r_f10', 'L'),('r_f15', 'L'),]



edges_list_2 =[('l_f1','l_f6'),('l_f1','l_f7'),('l_f6','l_f5'),('l_f6','l_f12'),('l_f6','l_f9'),('l_f7','l_f8'),('l_f12','l_f10'),
               ('l_f7','l_f8'),('l_f8','l_f3'),('l_f8','l_f13'),('l_f10','l_f4'),('l_f10','l_f2'),('l_f2','l_f11'),('l_f11','l_f14'),
               ('l_f14','l_f15'),('l_f4','L'),('l_f15','L'),('l_f13','L'),########
               ('r_f1','r_f6'),('r_f1','r_f7'),('r_f6','r_f5'),('r_f6','r_f12'),('r_f6','r_f9'),('r_f7','r_f8'),('r_f12','r_f10'),
               ('r_f7','r_f8'),('r_f8','r_f3'),('r_f8','r_f13'),('r_f10','r_f4'),('r_f10','r_f2'),('r_f2','r_f11'),('r_f11','r_f14'),
               ('r_f14','r_f15'),('r_f4','L'),('r_f15','L'),('r_f13','L')]


edges_list_3 =[('l_f3','l_f10'),('l_f4','l_f3'),('l_f4','l_f10'),('l_f4','l_f8'),('l_f8','l_f13'),('l_f13','l_f9'),('l_f10','l_f2'),
               ('l_f10','l_f11'),('l_f10','l_f13'),('l_f2','l_f12'),('l_f11','l_f12'),('l_f11','l_f14'),('l_f14','l_f15'),('l_f12','l_f7'),
               ('l_f12','l_f1'),('l_f1','l_f7'),('l_f7','l_f6'),('l_f1','l_f6'),('l_f6','l_f5'),('l_f5','L'),('l_f15','L'),('l_f9','L'),#####
               ('r_f3','r_f10'),('r_f4','r_f3'),('r_f4','r_f10'),('r_f4','r_f8'),('r_f8','r_f13'),('r_f13','r_f9'),('r_f10','r_f2'),
               ('r_f10','r_f11'),('r_f10','r_f13'),('r_f2','r_f12'),('r_f11','r_f12'),('r_f11','r_f14'),('r_f14','r_f15'),('r_f12','r_f7'),
               ('r_f12','r_f1'),('r_f1','r_f7'),('r_f7','r_f6'),('r_f1','r_f6'),('r_f6','r_f5'),('r_f5','L'),('r_f15','L'),('r_f9','L'),] 


edges_list_4 =[('l_f1','l_f12'),('l_f12','l_f2'),('l_f12','l_f6'),('l_f2','l_f3'),('l_f6','l_f5'),('l_f3','l_f11'),('l_f3','l_f8'),
               ('l_f3','l_f7'),('l_f5','l_f9'),('l_f11','l_f14'),('l_f11','l_f4'),('l_f8','l_f4'),('l_f14','l_f15'),('l_f4','l_f15'),
               ('l_f4','l_f10'),('l_f15','L'),('l_f10','L'),('l_f9','L'),###########################################################
               ('r_f1','l_f12'),('r_f12','l_f2'),('r_f12','r_f6'),('r_f2','r_f3'),('r_f6','r_f5'),('r_f3','r_f11'),('r_f3','r_f8'),
               ('r_f3','r_f7'),('r_f5','r_f9'),('r_f11','r_f14'),('r_f11','r_f4'),('r_f8','r_f4'),('r_f14','r_f15'),('r_f4','r_f15'),
               ('r_f4','r_f10'),('r_f15','L'),('r_f10','L'),('r_f9','L')]


edges_list_5 = [('l_f1','l_f2'),('l_f1','l_f6'),('l_f2','l_f5'),('l_f3','l_f2'),('l_f4','l_f6'),('l_f5','l_f9'),('l_f9','l_f7'),
                ('l_f9','l_f8'),("l_f7","l_f11"),("l_f8","l_f15"),('l_f6','l_f10'),('l_f10','l_f13'),('l_f10','l_f14'),('l_f14','l_f12'),
                ('l_f11','L'),('l_f15','L'),('l_f12','L'),##############################################################################
                ('r_f1','r_f2'),('r_f1','r_f6'),('r_f2','r_f5'),('r_f3','r_f2'),('r_f4','r_f6'),('r_f5','r_f9'),('r_f9','r_f7'),
                ('r_f9','l_f8'),("r_f7","r_f11"),("r_f8","r_f15"),('r_f6','r_f10'),('r_f10','r_f13'),('r_f10','r_f14'),('r_f14','r_f12'),
                ('r_f11','L'),('r_f15','L'),('r_f12','L')]
model_1 = model(nodes_list,edges_list_1,i)
i+=1
model_2 = model(nodes_list,edges_list_2,i)
i+=1
model_3 = model(nodes_list,edges_list_3,i)
i+=1
model_4 = model(nodes_list,edges_list_4,i)
i+=1
model_5 = model(nodes_list,edges_list_5,i)
i+=1

====Model1 is created====
====Model2 is created====
====Model3 is created====
====Model4 is created====
====Model5 is created====


In [0]:
start_time = time.time()
model_1.fit(data_hill_fit)
end_time = time.time()
print("The time taken to train model 1:",end_time-start_time)
print("model 1 is trained")
start_time = time.time()
model_2.fit(data_1)
end_time = time.time()
print("The time taken to train model 2:",end_time-start_time)
print("model 2 is trained")
start_time = time.time()
model_3.fit(data_1)
end_time = time.time()
print("The time taken to train model 3:",end_time-start_time)
print("model 3 is trained")
start_time = time.time()
model_4.fit(data_1)
end_time = time.time()
print("The time taken to train model 4:",end_time-start_time)
print("model 4 is trained")
start_time = time.time()
model_5.fit(data_1)
end_time = time.time()
print("The time taken to train model5:",end_time-start_time)


The time taken to train model 1: 151.7908170223236
model 1 is trained
The time taken to train model 2: 2.493422508239746
model 2 is trained
The time taken to train model 3: 2.545621633529663
model 3 is trained
The time taken to train model 4: 2.505708694458008
model 4 is trained
The time taken to train model5: 2.535381555557251


**K2 score**

In [0]:
k2 = K2Score(data_1)

In [0]:
print('Model 1 K2 Score: ' + str(k2.score(model_1)))
print('Model 2 K2 Score: ' + str(k2.score(model_2)))
print('Model 3 K2 Score: ' + str(k2.score(model_3)))
print('Model 4 K2 Score: ' + str(k2.score(model_4)))
print('Model 5 K2 Score: ' + str(k2.score(model_5)))

Model 1 K2 Score: -2065660.9476073976
Model 2 K2 Score: -2370194.6042688694
Model 3 K2 Score: -2300545.086716123
Model 4 K2 Score: -2414729.4009765973
Model 5 K2 Score: -2507631.9535876885


**TESTING**

In [0]:

def likelihood(data_evi,data_test,model):
  from tqdm import tqdm,tnrange, tqdm_notebook
  label_1, label_0  =[] ,[]
  time_b = []
  image2_keys  = (list(data_test))
  evi_keys     = np.asarray(list(data_evi))
  for row in tqdm_notebook( range(0,data_evi.shape[0])):
    v = np.asarray(data_evi.loc[row]).astype(int)
    evi_values = v-1
    evi_id = dict(zip(evi_keys, evi_values))
    ##=========================================
    image2_values = np.asarray(data_test.loc[row].astype(int))
    query_m = dict(zip(image2_keys,image2_values-1))
   # print(query_m)
    for i in range(0,2):
      evi_id.update(L = i )
     # print(evi_id)
      start_B = time.time()
      infer_B = VariableElimination(model)
     
      
       
      query = infer_B.query(variables= image2_keys , evidence= evi_id)
        
      
      end_B = time.time()
     
      diff_b = end_B -start_B
      time_b.append(diff_b)
      
      q = 1
      if (i == 0):
        v_0 = 1
        for key,value in query_m.items():
          #print(key)
          #print(value)
          q = query[key].values[value]
          v_0 = v_0 * q
        #print(v_0)
        label_0.append(v_0)
      else:
        v_1 =1
        for key,value in query_m.items():
          #print(key)
          #print(value)
          q = query[key].values[value]
          v_1 = v_1 * q
        #print(v_1)
        label_1.append(v_1)
  print("the time taken to infer in sec :",sum(time_b)) 
  
  return label_0,label_1    

def log_likelihood(v0,v1):
  predict_label =[]
  for i in range(0,len(v0)):
    import math
    v = v1[i]/v0[i]
    value = math.log(v)
    if (value>0.1):
      predict_label.append(1)
    else:
      predict_label.append(0)
  
  return predict_label
      
      
def accuracy(predict_label,data_labels):
  accuracy =[]
  for i in range(0,len(predict_label)):
    if(predict_label[i] ==data_labels[i] ):
      accuracy.append(1)
    else:
      accuracy.append(0)
  print("accuracy",(sum(accuracy)/len(predict_label))*100)
  
def save_ratioValues(v0,v1,filename):
  ratio_values = pd.DataFrame(
    {'v0': v0,
     'v1': v1, 
    })
  ratio_values.to_csv(filename+'.csv')
  print('filesaved',filename)
    
    

**Data Preprocessing**

1. Reading the validation set data which has 31 features.
2. Testing on only 50 pair  images  out of 906 pair images. 

    2.1. Taking 100 pair images of label = 0 and 100 pair images of label = 1.
3. Spliting the dataset into left [l_f1 to l_f15] and right [r_f1 to r_f15].

In [0]:
org_data_testing = pd.read_csv('merged_data_testing.csv')
org_data_testing = org_data_testing.dropna(how='any',axis=0)
data_testing_top = org_data_testing.loc[0:100]
data_testing_bottom = org_data_testing.loc[453:552]
data_frames = [data_testing_top,data_testing_bottom]
data_testing = pd.concat(data_frames,ignore_index=True)
print("Data with Label_1",data_testing_top.shape)
print("Data with Label 0",data_testing_bottom.shape)
print("Merged Data with label 1 and 0",data_testing.shape)
#
###==============================================##
data_evi = data_testing.loc[:,'l_f1':'l_f15']
print("features image1",data_evi.shape)
data_query = data_testing.loc[:,'r_f1':'r_f15']
print("features of image 2",data_query.shape)
data_testing
data_labels = data_testing.loc[:,'label']

Data with Label_1 (100, 34)
Data with Label 0 (100, 34)
Merged Data with label 1 and 0 (200, 34)
features image1 (200, 15)
features of image 2 (200, 15)


In [0]:
## Model_1 is the hill climb search model ##
v0,v1 = likelihood(data_evi,data_query,model_1)

The time taken for to infer is: 26.603769063949585
The time taken for to infer is: 27.41408658027649
The time taken for to infer is: 27.67840075492859
The time taken for to infer is: 27.576969146728516
The time taken for to infer is: 28.53686761856079
The time taken for to infer is: 28.73080015182495
The time taken for to infer is: 28.655043363571167
The time taken for to infer is: 28.195271730422974
The time taken for to infer is: 29.246232748031616
The time taken for to infer is: 28.547903299331665
The time taken for to infer is: 28.837741374969482
The time taken for to infer is: 28.83876371383667
The time taken for to infer is: 29.302558660507202
The time taken for to infer is: 29.714500665664673
The time taken for to infer is: 29.979969024658203
The time taken for to infer is: 38.2479944229126
The time taken for to infer is: 27.520915508270264
The time taken for to infer is: 27.3240647315979
The time taken for to infer is: 28.021663904190063
The time taken for to infer is: 27.24806

In [0]:
da = pd.read_csv('ratio_value_seen.csv')
v0 = da.loc[:,'v0'].tolist()
v1 = da.loc[:,'v1'].tolist()
len(v0)

200

In [1]:

save_ratioValues(v0,v1,'ratio_value_seen')
predict_label_seen = log_likelihood(v0,v1)
accuracy_seen = accuracy(predict_label_seen,data_labels)


accurcy 64.8


**Prediction for Unseen Dataset**

In [0]:
data_gen('dataset_unseen_validation_siamese','merged_unseen_testing_data')
org_data_testing = pd.read_csv('merged_unseen_testing_data.csv')
org_data_testing = org_data_testing.dropna(how='any',axis=0)
data_testing_top = org_data_testing.loc[0:50]
data_testing_bottom = org_data_testing.loc[3790:3844]
data_frames = [data_testing_top,data_testing_bottom]
data_testing = pd.concat(data_frames,ignore_index=True)
print("Data with Label_1",data_testing_top.shape)
print("Data with Label 0",data_testing_bottom.shape)
print("Merged Data with label 1 and 0",data_testing.shape)
#
###==============================================##
data_evi = data_testing.loc[:,'l_f1':'l_f15']
print("features image1",data_evi.shape)
data_query = data_testing.loc[:,'r_f1':'r_f15']
print("features of image 2",data_query.shape)
data_testing
data_labels = data_testing.loc[:,'label']


(13570, 18)
(7580, 4)
(7580, 21)
(7580, 37)
Data with Label_1 (50, 34)
Data with Label 0 (50, 34)
Merged Data with label 1 and 0 (100, 34)
features image1 (100, 15)
features of image 2 (100, 15)


In [0]:
uns_0,uns_1 = likelihood(data_evi,data_query,model_1)

99


In [2]:
save_ratioValues(uns_0,uns_1,'ratio_value_unseen')
predict_label_unseen = log_likelihood(uns_0,uns_1)
accuracy_unseen = accuracy(predict_label_unseen,data_labels)


accurcy 62.9999999999


**Prediction for Shuffled Dataset**

In [0]:
data_gen('dataset_shuffle_validation_siamese','merged_shuffle_testing_data')
org_data_testing = pd.read_csv('merged_unseen_testing_data.csv')
org_data_testing = org_data_testing.dropna(how='any',axis=0)
data_testing_top = org_data_testing.loc[0:49]
data_testing_bottom = org_data_testing.loc[3790:3840]
data_frames = [data_testing_top,data_testing_bottom]
data_testing = pd.concat(data_frames,ignore_index=True)
print("Data with Label_1",data_testing_top.shape)
print("Data with Label 0",data_testing_bottom.shape)
print("Merged Data with label 1 and 0",data_testing.shape)
#
###==============================================##
data_evi = data_testing.loc[:,'l_f1':'l_f15']
print("features image1",data_evi.shape)
data_query = data_testing.loc[:,'r_f1':'r_f15']
print("features of image 2",data_query.shape)
data_testing
data_labels = data_testing.loc[:,'label']

(13570, 18)
(5288, 4)
(5288, 21)
(5288, 37)
Data with Label_1 (50, 34)
Data with Label 0 (50, 34)
Merged Data with label 1 and 0 (100, 34)
features image1 (100, 15)
features of image 2 (100, 15)


In [0]:
sh_0,sh_1 = likelihood(data_evi,data_query,model_1)

the time taken to infer in sec : 5608.755061388016


In [4]:
save_ratioValues(sh_0,sh_1,'ratio_value_shuffle')
predict_label_shuffle = log_likelihood(sh_0,sh_1)
accuracy_shuffle = accuracy(predict_label_shuffle,data_labels)


accurcy 59.86458256


## ACCURACY 

1. For seen Dataset : 64.8 %
2. For Unseen Dataset : 62.9%
3. For Shuffle Dataset : 59.8 %

Time Taken To Infer:

1. For Seen Dataset         : 7200 sec approximately
2. For UnSeen Dataset    : 5400 sec approximately
3. For Shuffled Dataset   : 5608 sec approximately

#ROUGH

In [0]:
r_key1=(data_query.columns.values.tolist())
r_key  = np.asarray(list(data_query))
r_value = np.asarray(data_query.loc[0].astype(int))
#print(r_value)
dic = dict(zip(r_key,r_value))
#print(dic)
print(r_key1)

['r_f1', 'r_f2', 'r_f3', 'r_f4', 'r_f5', 'r_f6', 'r_f7', 'r_f8', 'r_f9', 'r_f10', 'r_f11', 'r_f12', 'r_f13', 'r_f14', 'r_f15']


In [0]:
i=0
var = ["r_f1" ,"r_f2","r_f3" ,"r_f4","r_f5" ,"r_f6","r_f7" ,"r_f8","r_f9","r_f10","r_f11","r_f12","r_f13","r_f14","r_f15"]
evi_keys = ['l_f1', 'l_f2', 'l_f3', 'l_f4', 'l_f5', 'l_f6', 'l_f7', 'l_f8', 'l_f9', 'l_f10', 'l_f11', 'l_f12', 'l_f13', 'l_f14', 'l_f15']
v = np.asarray(data_evi.loc[0]).astype(int)
evi_values = v
import itertools
infer_B = VariableElimination(model_1)
evid = dict(zip(evi_keys, evi_values))
evid.update(L = i )
#print(evid)
#query = infer_B.query(variables=["r_f1" ,"r_f2","r_f3" ,"r_f4","r_f5" ,"r_f6","r_f7" ,"r_f8","r_f9","r_f10","r_f11","r_f12","r_f13","r_f14","r_f15"],evidence= {'l_f1':v[0] ,'l_f2':v[1],'l_f3':v[2]-1,"l_f4":v[3],"l_f5":v[4] ,"l_f6":v[5],"l_f7":v[6] ,"l_f8":v[7],"l_f9":v[8],"l_f10":v[9],"l_f11":v[10],"l_f12":v[11],"l_f13":v[12],"l_f14":v[13],"l_f15":v[14],'L':1})
#query = infer_B.query(variables=var,evidence= evid)
#print(evid)
#query = infer_B.query(variables=["r_f1" ,"r_f2","r_f3" ,"r_f4","r_f5" ,"r_f6","r_f7" ,"r_f8","r_f9","r_f10","r_f11","r_f12","r_f13","r_f14","r_f15"],
#                      evidence= {'l_f1':1 ,'l_f2':3,'l_f3':1,'l_f4':1,'l_f5':1 })#"l_f6":1,"l_f7":2 ,"l_f8":2,"l_f9":1,"l_f10":2,"l_f11":2,"l_f12":2,"l_f13":3,"l_f14":1,"l_f15":1,'L':1})
#query = infer_B.query(variables=var,evidence= evid)
#likelihood(var,)
#1	3	3	3	2	1	2	2	1	4	2	2	3	1	1
print(v-1)

[0 2 2 2 1 0 0 0 0 1 1 0 0 1 1]


In [0]:
print(query['r_f1'])

+--------+-------------+
| r_f1   |   phi(r_f1) |
+========+=============+
| r_f1_0 |      0.4141 |
+--------+-------------+
| r_f1_1 |      0.5859 |
+--------+-------------+


In [0]:
import math
print('v_0',v_0)
print('v_1',v_1)
v = (math.log(v_1))/(math.log(v_0))
print(v)


v_0 0.0006236077295423567
v_1 0.0006565799279458211
0.9930185664779446


In [0]:
data_1.l_f1.unique()

array([1., 2.])